In [1]:
import os
import numpy as np
import pandas as pd
import csv

In [2]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
DATA_PATH = ROOT_PATH + '/data'
MYTRACK_PATH = ROOT_PATH + '/data/mytrack'

DATASET_MYTRACK_P = 'dataset_mytrack_points.pkl'
DATASET_MYTRACK_M = 'dataset_mytrack_markers.pkl'

os.chdir(ROOT_PATH)

In [3]:
# Carga del fichero de la aplicación MyTrack

def LoadMytrackCSV(s_file):
    
    l_lines = []
    s_pathfile = MYTRACK_PATH + '/' + s_file
    
    with open(s_pathfile,'r') as f:
        x = csv.reader(f)
        new = list(x)
        for i, line in enumerate(new):
            if (len(line) == 0):
                l_lines.append(i + 1)
        l_lines.append(len(new))
        f.close()
        
        if len(l_lines) >= 3:
            # El formato del fichero es correcto. Lo podemos cargar.
            df_points = pd.read_csv(s_pathfile, sep=',', skiprows=l_lines[1], encoding='UTF-8', engine='python')
            df_markers = pd.read_csv(s_pathfile, sep=',', skiprows=l_lines[0], skipfooter=l_lines[2] - l_lines[1], encoding='UTF-8', engine='python')
            df_markers.rename({'Name': 'Etiqueta'}, axis=1, inplace=True)
    
    return (df_points, df_markers)


In [4]:
def LoadMyTrack():
    
    # Obtengo los ficheros de MyTrack a cargar
    files_list = []
    for file in os.listdir(MYTRACK_PATH):
        if file.endswith(".csv"):
            files_list.append(file)

    df_points = pd.DataFrame(columns=['Location', 'Time', 'Latitude', 'Longitude', 'Altitude', 'Accuracy(m)', 'Speed(m/s)', 'Bearing', 'CodigoCultivo'])
    df_markers = pd.DataFrame(columns=['Marker', 'Time', 'Latitude', 'Longitude', 'Etiqueta', 'CodigoCultivo'])
    
    # Leo la información de los ficheros de MyTrack y la añado al dataframe
    for file in files_list:
        df_new_p, df_new_m = LoadMytrackCSV(file)
        
        s_tipo = file.split('-', 1)[0].upper()
        df_new_p['CodigoCultivo'] = s_tipo
        df_new_m['CodigoCultivo'] = s_tipo

        df_points = pd.concat([df_points, df_new_p], axis=0, ignore_index=True)
        df_markers = pd.concat([df_markers, df_new_m], axis=0, ignore_index=True)
    
    # Leo el fichero de configuración y completo los datos de MyTrack
    df_info = pd.read_csv(CONFIG_PATH + '/cultivos.cfg', sep=',', encoding='UTF-8', na_filter= False)
    
    df_points = pd.merge(df_points, df_info, on=['CodigoCultivo'], how='left')
    df_points.fillna('', inplace=True)
    df_points.drop(['CodigoCultivo'], axis=1, inplace=True)

    df_markers = pd.merge(df_markers, df_info, on=['CodigoCultivo'], how='left')
    df_markers.fillna('', inplace=True)
    df_markers.drop(['CodigoCultivo'], axis=1, inplace=True)
    
    return (df_points, df_markers)


In [5]:
# Función que guarda un dataset

def SaveDataset(df, name='dataset.pkl'):
    df.to_pickle(DATA_PATH + '/' + name)
 

In [6]:
# Función que fusiona la información del Catastro con la propia de la empresa 
# para generar el dataset que se visualizará postariormente

def GenerateDatasetMyTrack():
    # Cargamos los puntos de interés
    df_mytrack_points, df_mytrack_markers = LoadMyTrack()

    # Volcamos a disco los datasets
    SaveDataset(df_mytrack_points, DATASET_MYTRACK_P)
    SaveDataset(df_mytrack_markers, DATASET_MYTRACK_M)


In [7]:
# Función que carga el dataset de los markers

def LoadDatasetMyTrack():
    df_points = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_P)
    df_markers = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_M)
    return (df_points, df_markers)


## Ejecución

In [8]:
# Generación del dataset de puntos
GenerateDatasetMyTrack()